# Automated analysis of EEG quality

#### Loading data

In [21]:
## First let's load the training data
from pathlib import Path
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter
import pandas as pd

ROOT_PATH = Path("train")
training_data = [(np.load(ROOT_PATH / f"data_{i}.npy"),np.load(ROOT_PATH / f"target_{i}.npy")) for i in range(4)]

In [22]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    return butter(order, [lowcut, highcut], fs=fs, btype='band')

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [23]:
def reshape_array_into_windows(x, sample_rate, window_duration_in_seconds):
    """
    Reshape the data into an array of shape (C, T, window) where 'window' contains
    the points corresponding to 'window_duration' seconds of data.

    Parameters:
    x (numpy array): The input data array.
    sample_rate (int): The number of samples per second.
    window_duration_in_seconds (float): The duration of each window in seconds.

    Returns:
    reshaped_x (numpy array): The reshaped array with shape (C, T, window).
    """
    # Calculate the number of samples in one window
    window_size = int(window_duration_in_seconds * sample_rate)
    
    # Ensure the total length of x is a multiple of window_size
    total_samples = x.shape[-1]
    if total_samples % window_size != 0:
        # Truncate or pad x to make it divisible by window_size
        x = x[..., :total_samples - (total_samples % window_size)]
    # Reshape x into (C, T, window)
    reshaped_x = x.reshape(x.shape[0], -1, window_size)

    return reshaped_x

In [24]:
all_data = []
all_targets = []
for (data,target) in training_data:
    filtered_data =  butter_bandpass_filter(data,0.1,18,250,4)
    print(filtered_data.shape)
    reshaped_data = reshape_array_into_windows(filtered_data,250,2)
    print(reshaped_data.shape)
    targets_flatten = target[..., :len(reshaped_data[0])].reshape(-1)
    print(targets_flatten.shape)
    reshaped_data = reshaped_data.reshape((-1,reshaped_data.shape[-1]))
    print(reshaped_data.shape)
    all_data.append(reshaped_data)
    all_targets.append(targets_flatten)
all_data = np.concatenate(all_data)
all_targets = np.concatenate(all_targets)

(5, 7712740)
(5, 15425, 500)
(77125,)
(77125, 500)
(5, 5232364)
(5, 10464, 500)
(52320,)
(52320, 500)
(5, 6421756)
(5, 12843, 500)
(64215,)
(64215, 500)
(5, 6809761)
(5, 13619, 500)
(68095,)
(68095, 500)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import cohen_kappa_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import numpy as np

In [25]:
# We can now compute the mean, max and stdev over each 2 seconds segment to try to build features
amplitude = (np.max(all_data,-1) - np.min(all_data,-1)).reshape(-1)
maxwindow=(np.max(all_data,-1)).reshape(-1)

std = (np.std(all_data, axis=-1)).reshape(-1)
#df["log_std"] = np.log(std + 1)

training_data_features = pd.DataFrame({"amplitude":amplitude, "max":maxwindow, "std":std, "target":all_targets})

In [ ]:
# We train a model on 70% of the data and evaluate the model on the remaining 30%
prop_train = 0.7
n_train = int(prop_train * len(training_data_features))
train_set = training_data_features[:n_train]
val_set = training_data_features[n_train:]

neigh = KNeighborsClassifier(n_neighbors=8)
neigh.fit(np.array(train_set[["amplitude","std"]]), train_set["target"])
prediction = neigh.predict(np.array(val_set[["amplitude","std"]]))



print(cohen_kappa_score(prediction,val_set["target"]))
print(f1_score(val_set["target"],prediction))

0.5936568717279334
0.7331751824817518


In [ ]:


# Fraction pour l'entraînement
prop_train = 0.7

# Diviser les données de manière aléatoire en train et test
train_set, val_set = train_test_split(training_data_features, test_size=1 - prop_train, random_state=42)

# Initialiser le modèle KNN
neigh = KNeighborsClassifier(n_neighbors=8)

# Entraîner le modèle avec les colonnes "amplitude" et "std"
neigh.fit(np.array(train_set[["amplitude", "std"]]), train_set["target"])

# Faire des prédictions sur l'ensemble de validation
prediction = neigh.predict(np.array(val_set[["amplitude", "std"]]))



print(cohen_kappa_score(prediction,val_set["target"]))
print(f1_score(val_set["target"],prediction))

0.7342842408015843
0.8914599365011031


In [ ]:

#with random forest




prop_train = 0.7
n_train = int(prop_train * len(training_data_features))
train_set = training_data_features[:n_train]
val_set = training_data_features[n_train:]

rf = RandomForestClassifier(n_estimators=5, random_state=4)  # n_estimators = nombre d'arbres
rf.fit(np.array(train_set[["amplitude","std"]]), train_set["target"])
prediction = rf.predict(np.array(val_set[["amplitude","std"]]))


print(cohen_kappa_score(prediction,val_set["target"]))
print(f1_score(val_set["target"],prediction))

0.5497107727642652
0.7098133444769467


Nous allons maintenant tester de prendre en compte les 5 channels simultanément

In [32]:
all_data_bychannels = []
all_targets_bychannels = []
for (data,target) in training_data:
    filtered_data =  butter_bandpass_filter(data,0.1,18,250,4)
    print(filtered_data.shape)
    reshaped_data = reshape_array_into_windows(filtered_data,250,2)
    print(reshaped_data.shape)
    reshaped_data = reshaped_data.transpose(1, 0, 2)
    print(reshaped_data.shape)
    targets_flatten = target.transpose(1,0)
    print(targets_flatten.shape)
    all_data_bychannels.append(reshaped_data)
    all_targets_bychannels.append(targets_flatten)
all_data_bychannels = np.concatenate(all_data_bychannels)
all_targets_bychannels = np.concatenate(all_targets_bychannels)

print(all_targets_bychannels.shape)


(5, 7712740)
(5, 15425, 500)
(15425, 5, 500)
(15425, 5)
(5, 5232364)
(5, 10464, 500)
(10464, 5, 500)
(10464, 5)
(5, 6421756)
(5, 12843, 500)
(12843, 5, 500)
(12843, 5)
(5, 6809761)
(5, 13619, 500)
(13619, 5, 500)
(13619, 5)
(52351, 5)


In [39]:
amplitude = []
std = []
targets = []

block_size = 5  # Taille des blocs

# Parcourir les canaux de données
for channel_idx, channel_data in enumerate(all_data_bychannels):
    # Parcourir les segments de ce canal par blocs de 5
    for i in range(0, len(channel_data), block_size):
        block = channel_data[i:i + block_size]
        if len(block) == block_size:  # S'assurer que le bloc a la bonne taille
            # Calculer les caractéristiques pour chaque segment du bloc
            amp_block = [np.max(seg) - np.min(seg) for seg in block]
            std_block = [np.std(seg) for seg in block]
            
            # Ajouter les caractéristiques du bloc
            amplitude.append(amp_block)  # Liste des amplitudes des 5 segments
            std.append(std_block)  # Liste des écarts-types des 5 segments
            
            # Ajouter la cible correspondante (liste de 5 éléments)
            targets.append(all_targets_bychannels[channel_idx][i:i + block_size])

# Construire le DataFrame
training_data_features = pd.DataFrame({
    "amplitude": amplitude,
    "std": std,
    "target": targets
})

print(training_data_features.shape)

(52351, 3)


In [38]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import cohen_kappa_score, f1_score
import numpy as np
from sklearn.model_selection import train_test_split

# Diviser les données en ensemble d'entraînement et de validation
train_set, val_set = train_test_split(
    training_data_features, 
    test_size=0.3,  # 30% pour la validation
    random_state=42,  # Pour la reproductibilité
    shuffle=True  # Mélanger les données avant de les diviser
)

# Transformer les blocs de 5 en un tableau 2D pour les caractéristiques
X_train = np.hstack([
    np.stack(train_set["amplitude"].to_list()),  # Transforme chaque liste de 5 amplitudes en colonnes
    np.stack(train_set["std"].to_list())         # Transforme chaque liste de 5 std en colonnes
])
y_train = np.array(train_set["target"].to_list())  # Chaque cible reste un bloc de 5

X_val = np.hstack([
    np.stack(val_set["amplitude"].to_list()),
    np.stack(val_set["std"].to_list())
])
y_val = np.array(val_set["target"].to_list())

# Entraîner un modèle
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Faire des prédictions
predictions = rf.predict(X_val)

# Aplatir les cibles et les prédictions pour les métriques
y_val_flatten = y_val.flatten()
predictions_flatten = predictions.flatten()

# Calculer les scores
kappa = cohen_kappa_score(predictions_flatten, y_val_flatten)
f1 = f1_score(y_val_flatten, predictions_flatten)

print(f"Cohen Kappa Score: {kappa}")
print(f"F1 Score: {f1}")

Cohen Kappa Score: 0.8301405279671754
F1 Score: 0.9323029693537289


Le fait d'avoir pris en compte l'interaction entre les channels a fait augmenter le score Cohen Kappa.
Nous allons maintenant rajouter plusieurs features pour essayer d'améliorer notre modèle.

In [65]:
from scipy.signal import find_peaks
from scipy.signal import peak_prominences


def max_peak_prominence(x):
    peaks, _ = find_peaks(x)
    if len(peaks) == 0:
        return 0
    prominences = peak_prominences(x, peaks)[0]
    return np.max(prominences)

def count_peaks(x):
    return len(find_peaks(x)[0])

amplitude = []
std = []
MMD = []
energy = []
max_peak_prom = []
number_peaks = []
targets = []



block_size = 5  # Taille des blocs

# Parcourir les canaux de données
for channel_idx, channel_data in enumerate(all_data_bychannels):
    # Parcourir les segments de ce canal par blocs de 5
    for i in range(0, len(channel_data), block_size):
        block = channel_data[i:i + block_size]
        if len(block) == block_size:  # S'assurer que le bloc a la bonne taille
            # Calculer les caractéristiques pour chaque segment du bloc
            amp_block = [np.max(seg) - np.min(seg) for seg in block]
            std_block = [np.std(seg) for seg in block]
            MMD_block = [(np.argmax(seg)-np.argmin(seg))**2+(np.max(seg)-np.min(seg))**2 for seg in block]
            energy_block = [np.sum(seg**2) for seg in block]
            max_peak_prom_block = [max_peak_prominence(seg) for seg in block]
            number_peaks_block = [count_peaks(seg) for seg in block]

            # Ajouter les caractéristiques du bloc
            amplitude.append(amp_block)  # Liste des amplitudes des 5 segments
            std.append(std_block)  # Liste des écarts-types des 5 segments
            MMD.append(MMD_block)
            energy.append(energy_block)
            max_peak_prom.append(max_peak_prom_block)
            number_peaks.append(number_peaks_block)
            
            # Ajouter la cible correspondante (liste de 5 éléments)
            targets.append(all_targets_bychannels[channel_idx][i:i + block_size])

# Construire le DataFrame
training_data_features = pd.DataFrame({
    "amplitude": amplitude,
    "std": std,
    "MMD": MMD,
    "energy": energy,
    "max_peak_prom": max_peak_prom,
    "number_peaks": number_peaks,
    "target": targets
})

In [78]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import cohen_kappa_score, f1_score
import numpy as np
from sklearn.model_selection import train_test_split

# Diviser les données en ensemble d'entraînement et de validation
train_set, val_set = train_test_split(
    training_data_features, 
    test_size=0.3,  # 30% pour la validation
    random_state=42,  # Pour la reproductibilité
    shuffle=True  # Mélanger les données avant de les diviser
)

# Transformer les blocs de 5 en un tableau 2D pour les caractéristiques
X_train = np.hstack([
    np.stack(train_set["amplitude"].to_list()),  # Transforme chaque liste de 5 amplitudes en colonnes
    np.stack(train_set["std"].to_list()),         # Transforme chaque liste de 5 std en colonnes
    #np.stack(train_set["MMD"].to_list()),
    np.stack(train_set["energy"].to_list()),
    np.stack(train_set["max_peak_prom"].to_list()),
    np.stack(train_set["number_peaks"].to_list())
])
y_train = np.array(train_set["target"].to_list())  # Chaque cible reste un bloc de 5

X_val = np.hstack([
    np.stack(val_set["amplitude"].to_list()),
    np.stack(val_set["std"].to_list()),
    #np.stack(val_set["MMD"].to_list()),
    np.stack(val_set["energy"].to_list()),
    np.stack(val_set["max_peak_prom"].to_list()),
    np.stack(val_set["number_peaks"].to_list())
])
y_val = np.array(val_set["target"].to_list())

# Entraîner un modèle
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Faire des prédictions
predictions = rf.predict(X_val)
print(predictions.shape)

# Aplatir les cibles et les prédictions pour les métriques
y_val_flatten = y_val.flatten()
predictions_flatten = predictions.flatten()
print(predictions_flatten.shape)

# Calculer les scores
kappa = cohen_kappa_score(predictions_flatten, y_val_flatten)
f1 = f1_score(y_val_flatten, predictions_flatten)

print(f"Cohen Kappa Score: {kappa}")
print(f"F1 Score: {f1}")

(15706, 5)
(78530,)
Cohen Kappa Score: 0.9175809912163921
F1 Score: 0.9672132880811156


In [77]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer
import numpy as np
from sklearn.model_selection import train_test_split


# Diviser les données en ensemble d'entraînement et de validation
train_set, val_set = train_test_split(
    training_data_features, 
    test_size=0.3,  # 30% pour la validation
    random_state=42,  # Pour la reproductibilité
    shuffle=True  # Mélanger les données avant de les diviser
)

# Transformer les blocs de 5 en un tableau 2D pour les caractéristiques
X_train = np.hstack([
    np.stack(train_set["amplitude"].to_list()),  # Transforme chaque liste de 5 amplitudes en colonnes
    np.stack(train_set["std"].to_list()),         # Transforme chaque liste de 5 std en colonnes
    #np.stack(train_set["MMD"].to_list()),
    np.stack(train_set["energy"].to_list()),
    np.stack(train_set["max_peak_prom"].to_list()),
    np.stack(train_set["number_peaks"].to_list())
])
y_train = np.array(train_set["target"].to_list())  # Chaque cible reste un bloc de 5

X_val = np.hstack([
    np.stack(val_set["amplitude"].to_list()),
    np.stack(val_set["std"].to_list()),
    #np.stack(val_set["MMD"].to_list()),
    np.stack(val_set["energy"].to_list()),
    np.stack(val_set["max_peak_prom"].to_list()),
    np.stack(val_set["number_peaks"].to_list())
])
y_val = np.array(val_set["target"].to_list())



# Vérifier les dimensions des données
print("Shape of X_train:", X_train.shape)  # Exemple attendu : (36645, 25)
print("Shape of y_train:", y_train.shape)  # Exemple attendu : (36645, 5)

# Création du classificateur multi-sortie
multi_rf = MultiOutputClassifier(RandomForestClassifier(random_state=42))

# Définition des paramètres pour GridSearch
param_grid = {
    "estimator__n_estimators": [10, 50, 100],
    "estimator__max_depth": [None, 10, 20],
}

# Scorer basé sur F1-Score (calculé pour chaque label individuellement, puis moyenné)
scorer = make_scorer(f1_score, average="micro")

# Configuration de GridSearchCV
grid_search = GridSearchCV(
    multi_rf, 
    param_grid, 
    scoring=scorer, 
    cv=3,  # 3-fold cross-validation
    verbose=2,  # Affiche les informations d'entraînement
    n_jobs=-1   # Utilise tous les cœurs disponibles
)

# Entraîner avec GridSearch
grid_search.fit(X_train, y_train)

# Afficher les meilleurs paramètres et le score associé
print("Best parameters found:", grid_search.best_params_)
print("Best score (F1 micro):", grid_search.best_score_)

# Faire des prédictions sur l'ensemble de validation
predictions = grid_search.best_estimator_.predict(X_val)

# Vérifier les dimensions des prédictions
print("Shape of predictions:", predictions.shape)  # Exemple attendu : (nb_samples, 5)

# Calculer le F1-Score pour validation
f1_micro = f1_score(y_val, predictions, average="micro")
print("F1 Score on validation set (micro):", f1_micro)

f1_macro = f1_score(y_val, predictions, average="macro")
print("F1 Score on validation set (macro):", f1_macro)

Shape of X_train: (36645, 25)
Shape of y_train: (36645, 5)
Fitting 3 folds for each of 9 candidates, totalling 27 fits
Best parameters found: {'estimator__max_depth': 20, 'estimator__n_estimators': 100}
Best score (F1 micro): 0.9651361316262658
Shape of predictions: (15706, 5)
F1 Score on validation set (micro): 0.966914875613471
F1 Score on validation set (macro): 0.9614214792039208


In [79]:
# Entraîner le modèle avec les meilleurs paramètres
rf_final = RandomForestClassifier(n_estimators=100, random_state=42)
multi_rf_final = MultiOutputClassifier(rf_final)

# Entraîner sur l'ensemble d'entraînement complet
multi_rf_final.fit(X_train, y_train)

# Faire des prédictions sur l'ensemble de validation
predictions_final = multi_rf_final.predict(X_val)

# Calculer le Cohen Kappa Score final
from sklearn.metrics import cohen_kappa_score
kappa_final = cohen_kappa_score(y_val.flatten(), predictions_final.flatten())

print(f"Final Cohen Kappa Score: {kappa_final}")

import matplotlib.pyplot as plt

# Visualiser l'importance des caractéristiques
importances = rf_final.feature_importances_
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(10, 6))
plt.title("Feature Importances")
plt.bar(range(X_train.shape[1]), importances[indices], align="center")
plt.xticks(range(X_train.shape[1]), indices, rotation=90)
plt.xlim([-1, X_train.shape[1]])
plt.show()

Final Cohen Kappa Score: 0.9164015219254372


NotFittedError: This RandomForestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

Maintenant que le score est satisfaisant, il faut créer les fichiers csv pour les données de test et la soumission pour le leaderboard

In [67]:
ROOT_TEST_PATH = Path("test/")
test_data = {i:np.load(ROOT_TEST_PATH / f"data_{i}.npy") for i in [4,5]}
for key, value in test_data.items():
    print(f"Shape of data_{key}: {value.shape}")

Shape of data_4: (5, 6602015)
Shape of data_5: (5, 4659937)


In [69]:
# We process each record independantly

def compute_features_on_record(data):
    """
    We compute each of the feature for each window and each channel
    Each value of the output dict has shape (Channels,T)
    """
    filtered_data =  butter_bandpass_filter(data,0.1,18,250,4)
    reshaped_data = reshape_array_into_windows(filtered_data,250,2)
    reshaped_data = reshaped_data.transpose(1, 0, 2)
    amplitude = []
    std = []
    energy = []
    max_peak_prom = []
    number_peaks = []

    block_size = 5  # Taille des blocs

    # Parcourir les canaux de données
    for channel_idx, channel_data in enumerate(reshaped_data):
        # Parcourir les segments de ce canal par blocs de 5
        for i in range(0, len(channel_data), block_size):
            block = channel_data[i:i + block_size]
            if len(block) == block_size:  # S'assurer que le bloc a la bonne taille
                # Calculer les caractéristiques pour chaque segment du bloc
                amp_block = [np.max(seg) - np.min(seg) for seg in block]
                std_block = [np.std(seg) for seg in block]
                energy_block = [np.sum(seg**2) for seg in block]
                max_peak_prom_block = [max_peak_prominence(seg) for seg in block]
                number_peaks_block = [count_peaks(seg) for seg in block]

                # Ajouter les caractéristiques du bloc
                amplitude.append(amp_block)  # Liste des amplitudes des 5 segments
                std.append(std_block)  # Liste des écarts-types des 5 segments
                energy.append(energy_block)
                max_peak_prom.append(max_peak_prom_block)
                number_peaks.append(number_peaks_block)

    return pd.DataFrame({"amplitude":amplitude,
            "std": std,
            "energy": energy,
            "max_peak_prom": max_peak_prom,
            "number_peaks": number_peaks})

def compute_predictions_on_record(data,model,features_name_for_model):

    features = compute_features_on_record(data)
    

    X_test = np.hstack([
        np.stack(features["amplitude"].to_list()),
        np.stack(features["std"].to_list()),
        np.stack(features["energy"].to_list()),
        np.stack(features["max_peak_prom"].to_list()),
        np.stack(features["number_peaks"].to_list())
    ])
    
    predictions = model.predict(X_test)
    predictions = predictions.T
    return predictions

def format_array_to_target_format(array, record_number):
    assert isinstance(record_number, int)
    assert isinstance(array, np.ndarray)
    assert len(array.shape) == 2
    assert array.shape[0] == 5
    assert set(np.unique(array)) == {0, 1}
    formatted_target = []
    for i in range(array.shape[0]):
        channel_encoding = (i + 1) * 100000
        record_number_encoding = record_number * 1000000
        for j in range(array.shape[1]):
            formatted_target.append(
                {
                    "identifier": record_number_encoding + channel_encoding + j,
                    "target": array[i, j],
                }
            )
    return formatted_target



results = []
for record_number, data in test_data.items():
    preds = compute_predictions_on_record(data,rf,["amplitude","std","energy","max_peak_prom","number_peaks"])
    print(f"Shape of preds: {preds.shape}")
    formatted_preds = format_array_to_target_format(preds,record_number)
    results.extend(formatted_preds)
df = pd.DataFrame(results)
df.to_csv("submission_nesti5.csv",index = False)

Shape of preds: (5, 13204)
Shape of preds: (5, 9319)
